In [ ]:
import pandas as pd
import numpy as np
import csv

def csv_to_data_frame(file_name):
    titles = []
    datas = []

    with open(file_name, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                titles = row
                line_count += 1
            datas.append([int(value) if value.isnumeric() else value for (key, value) in row.items()])
            line_count += 1
        names = pd.DataFrame(datas, columns=titles)

        return names

In [ ]:
data = csv_to_data_frame("./NationalNames.csv")

In [ ]:
# task_3 Отримайте імена стовпців набору даних
data.columns

In [ ]:
#task_4 Отримайте загальну інформацію про дані у наборі даних
data[["Id", "Year", "Count"]].describe()

In [ ]:
#task_5 Знайдіть кількість унікальних імен у наборі даних
data["Name"].nunique()

In [ ]:
#task_8 Знайдіть найпопулярніше ім’я за результатами одного року (ім’я, для якого Count максимальне)
max_item = data.loc[data['Count'].idxmax()]
f"The name is '{max_item['Name']}' in {max_item['Year']}"

In [ ]:
#task_9 Підрахуйте кількість записів, для яких Count - мінімальне у наборі.
len(data.loc[data["Count"] == data["Count"].min()])

In [ ]:
#task_11 Знайдіть рік із найбільшою кількістю унікальних імен
unique = data.groupby("Year")["Name"].nunique()
popular_year = unique.idxmax()
unique.loc[popular_year:popular_year]

In [ ]:
#task_12 Знайдіть найпопулярніше ім’я в році з найбільшою кількістю унікальних імен (тобто у 2008 році)
popular_group = data.groupby("Year").get_group(popular_year)
popular_group.loc[popular_group['Count'].idxmax()].loc["Name"]

In [ ]:
#task_13 Знайдіть рік, коли ім’я “Jacob” було найпопулярнішим серед жіночих імен
group_name = data.groupby("Name")
jacob_group = group_name.get_group("Jacob")
jacob_f = jacob_group.loc[jacob_group["Gender"] == "F"]
f_index = jacob_f["Count"].idxmax()
jacob_f.loc[f_index:f_index]

In [ ]:
#task_14 Знайти рік із найбільшою кількістю гендерно нейтральних імен (однакові чоловічі та жіночі імена)
group = pd.DataFrame(data.groupby(["Year", "Gender"])["Name"].unique())
year_set = list(set([item[0] for item in group.index]))

m_array = group.loc[(year_set[0], "M")].values[0]
f_array = group.loc[(year_set[0], "F")].values[0]
max_count = len(np.intersect1d(m_array, f_array))
max_year = year_set[0]
for i in range(1, len(year_set)):
    year = year_set[i]
    m_array = group.loc[(year, "M")].values[0]
    f_array = group.loc[(year, "F")].values[0]
    count = len(np.intersect1d(m_array, f_array))
    if count > max_count:
        max_count = count
        max_year = year

pd.DataFrame([[max_year, max_count]], columns=["Year", "Gender_neutral_names"])

In [ ]:
#task_16 Знайдіть рік, коли народилося найбільше дітей
group = data.groupby(["Year"])["Count"].sum()
group.idxmax()

In [ ]:
#task_17 Знайдіть кількість дівчаток та хлопчиків, які народились кожного року

group_f = data.loc[data["Gender"] == "F"].groupby(["Year", "Gender"])["Count"].sum()
group_m = data.loc[data["Gender"] == "M"].groupby(["Year", "Gender"])["Count"].sum()

group_f_m = pd.merge(group_f, group_m, on=["Year"])
group_f_m.rename(columns={"Count_x": "F", "Count_y": "M"}, inplace=True)
group_f_m

In [ ]:
#task_18 Підрахуйте кількість років, коли дівчаток народжувалось більше, ніж хлопчиків

group_f_m.loc[group_f_m["F"] > group_f_m["M"]]["F"].count()

In [ ]:
#task_19 Накресліть графік загальної кількості народжень хлопчиків та дівчаток на рік.

group_f_m.plot()

In [ ]:
#task_20 Підрахуйте кількість гендерно-нейтральних імен (однакових для дівчат та хлопців)

group_f = data.loc[data["Gender"] == "F"]["Name"].unique()
group_m = data.loc[data["Gender"] == "M"]["Name"].unique()

len(np.intersect1d(group_f, group_m))

In [ ]:
#task_22 Підрахуйте скільки років проводилось спостереження
min_year = data["Year"].min()
max_year = data["Year"].max()
period = max_year - min_year + 1
f'Спостереження проводилося {period} років'

In [ ]:
#task_23 Знати найпопулярніші гендерно-нейтральні імена (ті, що присутні кожного року)

def get_unique_names(group_):
    res = np.array([])
    for j in range(0, len(group_)):
        if len(res) == 0:
            res = group_.iloc[j]
        else:
            res = np.intersect1d(res, group_.iloc[j])

    return res


group_f = data.loc[data["Gender"] == "F"].groupby(["Year"])["Name"].unique()
group_m = data.loc[data["Gender"] == "M"].groupby(["Year"])["Name"].unique()
unique_f = get_unique_names(group_f)
unique_m = get_unique_names(group_m)
pd.Series(np.intersect1d(unique_f, unique_m))

In [ ]:
#task_25 Побудувати графіки розподілення кількості імен John та Mary по роках без залежності до статі

group_f = data.loc[data["Name"] == "John"].groupby(["Year"])["Count"].sum()
group_m = data.loc[data["Name"] == "Mary"].groupby(["Year"])["Count"].sum()

group_f_m = pd.merge(group_f, group_m, on=["Year"])
group_f_m.rename(columns={"Count_x": "John", "Count_y": "Mary"}, inplace=True)
group_f_m.plot()

In [ ]:
#task_27 Знайти найпопулярніші імена в кожному році

maxes = data.groupby(["Year"])["Count"].idxmax()
data.iloc[maxes.values].groupby(["Year"])[["Name", "Count"]].max()